In [54]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from single_camera_calibrator import *
cv2.__version__

'3.4.2'

In [55]:
original_left_image = cv2.imread('./images/left/left03.jpg')  
original_right_image = cv2.imread('./images/right/right03.jpg')
undistorted_left_image = cv2.imread('./images/undistorted_left/left03.jpg')  
undistorted_right_image = cv2.imread('./images/undistorted_right/right03.jpg')
rectified_left_image = cv2.imread('./images/rectify_left/left03.jpg')  
rectified_right_image = cv2.imread('./images/rectify_right/right03.jpg')

In [49]:

plt.subplot(3,2,1)
plt.imshow(original_left_image)
plt.title('original left image')
plt.subplot(3,2,2)
plt.imshow(original_right_image)
plt.title('original right image')
plt.subplot(3,2,3)
plt.imshow(undistorted_left_image)
plt.title('left image after undistortion')
plt.subplot(3,2,4)
plt.imshow(original_right_image)
plt.title('right image undistortion')
plt.subplot(3,2,5)
plt.imshow(left_image)
plt.title('left image after rectify')
plt.subplot(3,2,6)
plt.imshow(right_image)
plt.title('right image after rectify')
plt.show()


gray_left_image = cv2.cvtColor(left_image, cv2.COLOR_BGR2GRAY)
gray_right_image = cv2.cvtColor(right_image, cv2.COLOR_BGR2GRAY)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [10]:
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(gray_left_image,None)
kp2, des2 = sift.detectAndCompute(gray_right_image,None)

featured_left_image = cv2.drawKeypoints(left_image,kp1,left_image,color=(255,0,255))
featured_right_image = cv2.drawKeypoints(right_image,kp2,right_image,color=(255,0,255))

plt.subplot(1,2,1)
plt.imshow(featured_left_image)
plt.subplot(1,2,2)
plt.imshow(featured_right_image)
plt.show()

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

# FLANNBasedMatcher
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
good_matches = []
left_image_points = []
right_image_points = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):

    if m.distance < 0.7*n.distance:
        good_matches.append(m)
        right_image_points.append(kp2[m.trainIdx].pt)
        left_image_points.append(kp1[m.queryIdx].pt)
        
matched_image = cv2.drawMatches(left_image, kp1, right_image, kp2, good_matches, None, flags=2)

plt.imshow(matched_image)
plt.show()


In [50]:

left_image_points = np.int32(left_image_points)
right_image_points = np.int32(right_image_points)
F, mask = cv2.findFundamentalMat(
    points1=left_image_points,
    points2=right_image_points,
    method=cv2.FM_LMEDS
)


In [51]:
# We select only inlier points
left_image_points = left_image_points[mask.ravel()==1]
right_image_points = right_image_points[mask.ravel()==1]


In [52]:
def drawlines(image,lines,points):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = image.shape[:2]

    for r,point in zip(lines,points):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        image = cv2.line(image, (x0,y0), (x1,y1), color,1)
        image = cv2.circle(image,tuple(point),5,color,-1)
        
    return image

In [53]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(right_image_points.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)

epipolar_left_image = drawlines(
    left_image,
    lines1,
    left_image_points,
)


# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(left_image_points.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)

epipolar_right_image = drawlines(
    right_image,
    lines2,
    right_image_points,
)

plt.subplot(1,2,1)
plt.imshow(epipolar_left_image)
plt.subplot(1,2,2)
plt.imshow(epipolar_right_image)
plt.show()